In [1]:
import numpy as np
import pandas as pd
import dtale
import pandas_profiling as pp
from IPython.display import Javascript
from plotly.offline import iplot, init_notebook_mode

In [2]:
# load the dataset
accident_victims_2019_df = pd.read_parquet("../../data/silver/accident_victims_2019_df.parquet")

In [3]:
accident_victims_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85607 entries, 0 to 85606
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DT_DAY                     85607 non-null  datetime64[ns]
 1   DT_HOUR                    85607 non-null  Int64         
 2   CD_DAY_OF_WEEK             85607 non-null  Int64         
 3   TX_DAY_OF_WEEK_DESCR_FR    85607 non-null  string        
 4   TX_DAY_OF_WEEK_DESCR_NL    85607 non-null  string        
 5   MS_VICT                    85607 non-null  Int64         
 6   MS_VIC_OK                  85607 non-null  Int64         
 7   MS_SLY_INJ                 85607 non-null  Int64         
 8   MS_SERLY_INJ               85607 non-null  Int64         
 9   MS_DEAD_30_DAYS            85607 non-null  Int64         
 10  CD_BUILD_UP_AREA           85607 non-null  Int64         
 11  TX_BUILD_UP_AREA_DESCR_NL  85607 non-null  string        
 12  TX_B

In [4]:
# remove any duplicate values
accident_victims_2019_df = accident_victims_2019_df.drop_duplicates().reset_index(drop=True)

# keep only the victim type 'bicycle' users

In [5]:
accident_victims_2019_df['TX_VICT_TYPE_DESCR_NL'].unique()

<StringArray>
[    'Bromfietser',      'Bestuurder',      'Voetganger',       'Passagier',
         'Fietser',    'Motorfietser', 'Autres victimes',        'Onbekend']
Length: 8, dtype: string

In [6]:
# remove all other records, keep only the victim type 'bicycle' type 
accident_victims_2019_df = accident_victims_2019_df[(accident_victims_2019_df['TX_VICT_TYPE_DESCR_NL'] == 'Bestuurder')]

In [7]:
# select only a few important columns (date, vilage, ...)
accident_per_day_2019_df = accident_victims_2019_df[["DT_DAY", "TX_BUILD_UP_AREA_DESCR_NL", "CD_ROAD_USER_TYPE", "TX_ROAD_TYPE_DESCR_NL", "TX_LIGHT_COND_DESCR_NL", "TX_AGE_CLS_DESCR_NL", "TX_MUNTY_DESCR_NL"]]

In [8]:
accident_per_day_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49315 entries, 1 to 85583
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DT_DAY                     49315 non-null  datetime64[ns]
 1   TX_BUILD_UP_AREA_DESCR_NL  49315 non-null  string        
 2   CD_ROAD_USER_TYPE          49315 non-null  Int64         
 3   TX_ROAD_TYPE_DESCR_NL      49315 non-null  string        
 4   TX_LIGHT_COND_DESCR_NL     49315 non-null  string        
 5   TX_AGE_CLS_DESCR_NL        49315 non-null  string        
 6   TX_MUNTY_DESCR_NL          49315 non-null  string        
dtypes: Int64(1), datetime64[ns](1), string(5)
memory usage: 3.1 MB


In [9]:
accident_per_day_2019_df

,DT_DAY,TX_BUILD_UP_AREA_DESCR_NL,CD_ROAD_USER_TYPE,TX_ROAD_TYPE_DESCR_NL,TX_LIGHT_COND_DESCR_NL,TX_AGE_CLS_DESCR_NL,TX_MUNTY_DESCR_NL
1,2019-12-15,Binnen bebouwde kom,1,Gewestweg,Bij klaarlichte dag,30 tot 34 jaar,Aartselaar
2,2019-12-30,Onbekend,1,Gemeenteweg,Bij klaarlichte dag,Onbekend,Aartselaar
5,2019-12-11,Buiten bebouwde kom,2,Gemeenteweg,"Nacht, ontstoken openbare verlichting",40 tot 44 jaar,Aartselaar
10,2019-11-19,Buiten bebouwde kom,1,Gewestweg,Bij klaarlichte dag,25 tot 29 jaar,Aartselaar
11,2019-11-27,Binnen bebouwde kom,1,Gemeenteweg,Bij klaarlichte dag,35 tot 39 jaar,Aartselaar
...,...,...,...,...,...,...,...
85577,2019-06-15,Buiten bebouwde kom,1,Gewestweg,Bij klaarlichte dag,65 tot 69 jaar,Viroinval
85578,2019-07-02,Binnen bebouwde kom,2,Gemeenteweg,Bij klaarlichte dag,60 tot 64 jaar,Viroinval
85580,2019-09-03,Binnen bebouwde kom,4,Gemeenteweg,Bij klaarlichte dag,75 jaar en meer,Viroinval
85582,2019-09-16,Binnen bebouwde kom,1,Gewestweg,Bij klaarlichte dag,30 tot 34 jaar,Viroinval


## count is the right function to use
but, we need the number of incidents that day

In [10]:
# sort date first, then sort on vilagename, then group these two and count() the number of incidents that occured
# then create a new dataframe and sort the results with the most accidents on a given day first 
groupby_accident_per_day_2019_df = accident_per_day_2019_df.groupby(['DT_DAY', 'TX_MUNTY_DESCR_NL'])['TX_ROAD_TYPE_DESCR_NL'].count().to_frame(name='nr_of_accidents').reset_index()
groupby_accident_per_day_2019_df = groupby_accident_per_day_2019_df.sort_values(by=['nr_of_accidents', 'DT_DAY', 'TX_MUNTY_DESCR_NL'], ascending=[False, True, False])
groupby_accident_per_day_2019_df

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents
4921,2019-03-18,Antwerpen,25
6047,2019-04-03,Antwerpen,24
5977,2019-04-02,Antwerpen,23
24652,2019-12-20,Antwerpen,23
10985,2019-06-14,Antwerpen,22
...,...,...,...
25245,2019-12-31,Hechtel-Eksel,1
25237,2019-12-31,Borsbeek,1
25233,2019-12-31,Awans,1
25232,2019-12-31,Avelgem,1


In [11]:
# this was to check and verify the correctness of the results - ALL GOOD
groupby_accident_per_day_2019_df[groupby_accident_per_day_2019_df["TX_MUNTY_DESCR_NL"] == 'Antwerpen'].sort_values(by=['DT_DAY'], ascending=[True])

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents
1,2019-01-01,Antwerpen,6
56,2019-01-02,Antwerpen,7
104,2019-01-03,Antwerpen,4
148,2019-01-04,Antwerpen,11
187,2019-01-05,Antwerpen,7
...,...,...,...
25052,2019-12-27,Antwerpen,9
25101,2019-12-28,Antwerpen,6
25140,2019-12-29,Antwerpen,2
25172,2019-12-30,Antwerpen,5


# so now, we calculate the accidents/cyclists RATIO

In [12]:
# load population values
village_populations_wiki_df = pd.read_parquet("../../data/silver/village_populations_wiki_df.parquet")

In [13]:
# village_populations_wiki_df.columns = ["Village", "Population", "km2_size"]

In [14]:
village_populations_wiki_df

,Village,Population,km2_size
0,Antwerpen,446525,"204,51"
1,Gent,224180,"156,18"
2,Charleroi,200827,"102,08"
3,Luik,185639,"69,39"
4,Brussel,133859,"32,61"
...,...,...,...
576,Martelange,1428,"29,67"
577,Herbeumont,1434,"58,81"
578,Daverdisse,1360,"56,40"
579,Mesen,964,"3,58"


In [15]:
village_populations_wiki_df['km2_size'] = village_populations_wiki_df['km2_size'].str[:-3:].astype(int)

In [16]:
village_populations_wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Village     581 non-null    string
 1   Population  581 non-null    Int64 
 2   km2_size    581 non-null    int64 
dtypes: Int64(1), int64(1), string(1)
memory usage: 14.3 KB


# now we calculate the ratio

In [17]:
groupby_accident_per_day_2019_df = groupby_accident_per_day_2019_df.replace({'TX_MUNTY_DESCR_NL':{
    'Sint-Niklaas (Sint-Niklaas)': 'Sint-Niklaas',
    'Halle (Halle-Vilvoorde)': 'Halle',
    'Beveren (Sint-Niklaas)': 'Beveren',
    'Tielt (Tielt)': 'Tielt',
    'Vorst (Brussel-Hoofdstad)': 'Vorst',
    'Kapellen (Antwerpen)': 'Kapellen',
    'Hamme (Dendermonde)': 'Hamme',
    'Nieuwerkerken (Hasselt)': 'Nieuwerkerken',
    'Machelen (Halle-Vilvoorde)': 'Machelen',
    'Hove (Antwerpen)': 'Hove',
    'Celles (Doornik)': 'Celles',
    'Perwijs (Nijvel)': 'Perwijs',
    'Saint-Nicolas (Luik)': 'Saint-Nicolas',
    'Aalst (Aalst)': 'Aalst',
    'Moerbeke (Gent)': 'Moerbeke'}})

In [18]:
village_populations_wiki_df = village_populations_wiki_df.replace({'Village':{
    'Pelt*': 'Pelt',
    'Deinze*': 'Deinze',
    'Kruisem*': 'Kruisem',
    'Aalter*': 'Aalter',
    'Belœil': 'Beloeil',
    'Lievegem*': 'Lievegem',
    "Mont-de-l'Enclus": 'Mont-de-l’Enclus',
    'Puurs-Sint-Amands*': 'Puurs-Sint-Amands',
    'Court-Saint-Étienne': 'Court-Saint-Etienne',
    'Éghezée': 'Eghezée',
    'Oudsbergen*': 'Oudsbergen',
    'Estaimpuis*': 'Estaimpuis',
    "Fontaine-l'Evêque": 'Fontaine-l’Evêque',
    'Gembloers': 'Gembloux',
    '’s Gravenbrakel*': '’s Gravenbrakel',
    'Erezée*': 'Erezée',
    'Ecaussines': 'Ecaussinnes',
    'Blegny*': 'Blegny',
    'Plombières*': 'Plombières'}})

In [19]:
# merge join on village
merged_accidents_df = groupby_accident_per_day_2019_df.merge(village_populations_wiki_df, left_on='TX_MUNTY_DESCR_NL', right_on='Village', how='left')

In [20]:
# remove Nan records
merged_accidents_df = merged_accidents_df[~(merged_accidents_df['Population'].isna())]

# calculate the ratio itself

### (nr of incidents / population ) * city size in km

In [21]:
merged_accidents_df.loc[:, 'accident_village_ratio'] = merged_accidents_df['nr_of_accidents'] / merged_accidents_df['Population']

In [22]:
merged_accidents_df.loc[:, 'accident_village_size_ratio'] = merged_accidents_df['accident_village_ratio'] * merged_accidents_df['km2_size']

In [23]:
merged_accidents_df

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents,Village,Population,km2_size,accident_village_ratio,accident_village_size_ratio
0,2019-03-18,Antwerpen,25,Antwerpen,446525,204.0,0.000056,0.011422
1,2019-04-03,Antwerpen,24,Antwerpen,446525,204.0,0.000054,0.010965
2,2019-04-02,Antwerpen,23,Antwerpen,446525,204.0,0.000052,0.010508
3,2019-12-20,Antwerpen,23,Antwerpen,446525,204.0,0.000052,0.010508
4,2019-06-14,Antwerpen,22,Antwerpen,446525,204.0,0.000049,0.010051
...,...,...,...,...,...,...,...,...
25270,2019-12-31,Hechtel-Eksel,1,Hechtel-Eksel,11149,76.0,0.00009,0.006817
25271,2019-12-31,Borsbeek,1,Borsbeek,10373,3.0,0.000096,0.000289
25272,2019-12-31,Awans,1,Awans,8270,27.0,0.000121,0.003265
25273,2019-12-31,Avelgem,1,Avelgem,9148,21.0,0.000109,0.002296


In [24]:
accident_village_ratio_median_cutoff = merged_accidents_df['accident_village_ratio'].median()
accident_village_ratio_median_cutoff

6.444752360390551e-05

In [25]:
accident_village_size_ratio_median_cutoff = merged_accidents_df['accident_village_size_ratio'].median()
accident_village_size_ratio_median_cutoff

0.003054870168017859

In [27]:
result = merged_accidents_df.groupby(['Village'])['accident_village_size_ratio'].sum().to_frame(name='ratio').reset_index()
result = result.sort_values(by=['ratio', 'Village'], ascending=[False, True])
result.head(10)

,Village,ratio
332,Léglise,1.766009
19,Antwerpen,1.450078
341,Manhay,1.237435
38,Beauraing,1.225049
98,Chimay,1.15063
132,Doornik,1.147524
240,Houffalize,1.106421
56,Beveren,1.100563
412,Philippeville,1.10039
108,Couvin,1.088384


In [28]:
s = result['ratio']
q1 = s.quantile(0.25)
q3 = s.quantile(0.75)
iqr = q3 - q1
iqr_lower = q1 - 1.5 * iqr
iqr_upper = q3 + 1.5 * iqr
outliers = dict(s[(s < iqr_lower) | (s > iqr_upper)])

In [29]:
result.iloc[list(outliers.keys())]

,Village,ratio
262,Kampenhout,0.144521
474,Somme-Leuze,0.797362
69,Borgloon,0.141163
511,Veurne,0.60739
85,Brussel,0.389664
215,Heist-op-den-Berg,0.321059
253,Itter,0.204566
516,Virton,0.50377
58,Bierbeek,0.106098
111,Damme,0.37023


In [ ]:
result.iloc[list(outliers.keys())]

In [30]:
chart_data = pd.concat([
    result.iloc[list(outliers.keys())]['Village'],
    result.iloc[list(outliers.keys())]['ratio'],
], axis=1)
chart_data = chart_data.sort_values(['Village'])
chart_data = chart_data.rename(columns={'Village': 'x'})
chart_data = chart_data.dropna()

import plotly.graph_objs as go

charts = []
charts.append(go.Bar(
    x=chart_data['x'],
    y=chart_data['ratio']
))
figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'stack',
    'legend': {'orientation': 'h'},
    'title': {'text': 'Outliers (highest & lowest)'},
    'xaxis': {'title': {'text': 'Cities'}},
    'yaxis': {'title': {'text': 'ratio of cars'}, 'type': 'linear'}
}))

In [34]:
init_notebook_mode(connected=True)
for chart in charts:
    chart.pop('id', None) # for some reason iplot does not like 'id'
iplot(figure)

In [33]:
figure.write_image("../../data/plot/accident_ratio_cars.png")